In [4]:
# Install required packages if not installed in your local machine.
#Install this Missing dependency or esle
#!pip install python-dotenv openai pandas

# 1. Configuration
import os
from dotenv import load_dotenv
from getpass import getpass
import json
from openai import OpenAI
from IPython.display import display, Markdown
import pandas as pd

def get_api_key():
    """Safely load API key from .env or user input"""
    load_dotenv()
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        print("🔐 Paste your OpenAI API key (hidden input):")
        api_key = getpass("").strip()
        if not api_key.startswith("sk-"):
            raise ValueError("Invalid API key format - must start with 'sk-'")
        with open('.env', 'w') as f:
            f.write(f"OPENAI_API_KEY={api_key}\n")
    return api_key

# 2. Class Definition
class InsightSynthesizer:
    def __init__(self, model="gpt-3.5-turbo"):
        self.client = OpenAI(api_key=get_api_key())
        self.model = model
        self.prompt_template = """Analyze these survey responses:
        {responses}
        
        Return JSON with structure:
        {{
          "themes": [
            {{
              "theme": "3-5 word summary",
              "quotes": ["representative quote"],
              "sentiment": "positive/neutral/negative"
            }}
          ]
        }}"""
    
    def generate_insights(self, responses):
        try:
            processed = "\n".join(f"- {r}" for r in responses)
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "You are a helpful research assistant that outputs valid JSON."},
                    {"role": "user", "content": self.prompt_template.format(responses=processed)}
                ],
                temperature=0.3,
                response_format={"type": "json_object"}
            )
            return json.loads(response.choices[0].message.content)["themes"]
        except Exception as e:
            print(f"⚠️ Error: {e}")
            return []

# 3. Analysis
if __name__ == "__main__":
    # Remove old .env if exists
    if os.path.exists('.env'):
        os.remove('.env')
    
    analyzer = InsightSynthesizer()
    
    responses = [
        "The mobile app crashes frequently",
        "Customer support was very helpful",
        "Would love more customization options",
        "Too expensive for the current features"
    ]
    
    results = analyzer.generate_insights(responses)
    
    if results:
        display(Markdown("### 📊 Insights"))
        df = pd.DataFrame(results)
        display(df)
        
        with open('insights.json', 'w') as f:
            json.dump(results, f, indent=2)
        display(Markdown("✅ Saved to `insights.json`"))
        
        display(Markdown("### 💬 Key Quotes"))
        for i, theme in enumerate(results, 1):
            display(Markdown(f"{i}. **{theme['theme']}** ({theme['sentiment']})"))
            for q in theme['quotes']:
                display(Markdown(f"   - \"{q}\""))
    else:
        display(Markdown("❌ No results generated"))

🔐 Paste your OpenAI API key (hidden input):


 ········


⚠️ Error: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


❌ No results generated